In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Creativity/Starling_7b_16_FT_WritingPrompts_full_qkv.xlsx'
df = pd.read_excel(file_path, sheet_name ='Sheet1', engine='openpyxl')

In [ ]:
model_path = '/content/drive/MyDrive/Creativity/Starling_7b_ft_WritingPrompts_full_qkv'

In [ ]:
# Load the model and tokenizer
# model_name = "berkeley-nest/Starling-LM-7B-alpha"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)
pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

In [ ]:
def generate_response(prompt, n=1 ,max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    responses = []
    for _ in range(n):
        with torch.no_grad():
            output = model.generate(**inputs,
                         max_length=max_length,
                         do_sample=True,
                         top_k=50,
                         top_p=0.9,
                         temperature=1.0,
                         pad_token_id=pad_token_id)
            response = tokenizer.decode(output[0], skip_special_tokens=True)
            response = response[len(prompt):].strip()
        responses.append(response)
    return responses

In [ ]:
print(df.columns)

Index(['Prompts', 'Response_1', 'Response_2', 'Response_3', 'Functionality',
       'Fluency', 'Flexibility', 'Originality', 'Elaboration'],
      dtype='object')


In [ ]:
# Ensure the columns can hold string data by converting them to string type
df['Response_1'] = df['Response_1'].astype(str)
df['Response_2'] = df['Response_2'].astype(str)
df['Response_3'] = df['Response_3'].astype(str)

In [ ]:
# Generate responses for each prompt and add them to the DataFrame
for idx, row in df.iterrows():
    prompt = row['Prompts']
    responses = generate_response(prompt)
    df.at[idx, 'Response_1'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_2'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_3'] = responses[0]

In [ ]:
# Save the updated DataFrame back to Google Drive
updated_file_path = '/content/drive/MyDrive/Creativity/Starling_7b_16_FT_WritingPrompts_full_qkv.xlsx'
df.to_excel(updated_file_path, index=False, engine='openpyxl')